## 1. The NIST Special Publication 800-63B
<p>If you – 50 years ago – needed to come up with a secret password you were probably part of a secret espionage organization or (more likely) you were pretending to be a spy when playing as a kid. Today, many of us are forced to come up with new passwords <em>all the time</em> when signing into sites and apps. As a password <em>inventeur</em> it is your responsibility to come up with good, hard-to-crack passwords. But it is also in the interest of sites and apps to make sure that you use good passwords. The problem is that it's really hard to define what makes a good password. However, <em>the National Institute of Standards and Technology</em> (NIST) knows what the second best thing is: To make sure you're at least not using a <em>bad</em> password. </p>
<p>In this notebook, we will go through the rules in <a href="https://pages.nist.gov/800-63-3/sp800-63b.html">NIST Special Publication 800-63B</a> which details what checks a <em>verifier</em> (what the NIST calls a second party responsible for storing and verifying passwords) should perform to make sure users don't pick bad passwords. We will go through the passwords of users from a fictional company and use python to flag the users with bad passwords. But us being able to do this already means the fictional company is breaking one of the rules of 800-63B:</p>
<blockquote>
  <p>Verifiers SHALL store memorized secrets in a form that is resistant to offline attacks. Memorized secrets SHALL be salted and hashed using a suitable one-way key derivation function.</p>
</blockquote>
<p>That is, never save users' passwords in plaintext, always encrypt the passwords! Keeping this in mind for the next time we're building a password management system, let's load in the data.</p>
<p><em>Warning: The list of passwords and the fictional user database both contain <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have not been filtered in any way and include words that are explicit, derogatory and offensive.</em></p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Importing the pandas module
# ... YOUR CODE FOR TASK 1 ...

# Loading in datasets/users.csv 
users = pd.read_csv('./datasets/users.csv')

In [3]:
# Printing out how many users we've got
# ... YOUR CODE FOR TASK 1 ...

users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 3 columns):
id           982 non-null int64
user_name    982 non-null object
password     982 non-null object
dtypes: int64(1), object(2)
memory usage: 23.1+ KB


In [4]:
# Taking a look at the 12 first users
# ... YOUR CODE FOR TASK 1 ...

users.head(n=12)

,id,user_name,password
0,1,vance.jennings,joobheco
1,2,consuelo.eaton,0869347314
2,3,mitchel.perkins,fabypotter
3,4,odessa.vaughan,aharney88
4,5,araceli.wilder,acecdn3000
5,6,shawn.harrington,5278049
6,7,evelyn.gay,master
7,8,noreen.hale,murphy
8,9,gladys.ward,lwsves2
9,10,brant.zimmerman,1190KAREN5572497


## 2. Passwords should not be too short
<p>If we take a look at the first 12 users above we already see some bad passwords. But let's not get ahead of ourselves and start flagging passwords <em>manually</em>. What is the first thing we should check according to the NIST Special Publication 800-63B?</p>
<blockquote>
  <p>Verifiers SHALL require subscriber-chosen memorized secrets to be at least 8 characters in length.</p>
</blockquote>
<p>Ok, so the passwords of our users shouldn't be too short. Let's start by checking that!</p>

In [7]:
# Calculating the lengths of users' passwords
users['password_length'] = users['password'].apply(lambda x: len(x))

In [15]:
users.head()

,id,user_name,password,password_length
0,1,vance.jennings,joobheco,8
1,2,consuelo.eaton,0869347314,10
2,3,mitchel.perkins,fabypotter,10
3,4,odessa.vaughan,aharney88,9
4,5,araceli.wilder,acecdn3000,10


In [17]:
# Flagging the users with too short passwords
users['too_short'] = users['password_length'] < 8 
users.head()

,id,user_name,password,password_length,too_short
0,1,vance.jennings,joobheco,8,False
1,2,consuelo.eaton,0869347314,10,False
2,3,mitchel.perkins,fabypotter,10,False
3,4,odessa.vaughan,aharney88,9,False
4,5,araceli.wilder,acecdn3000,10,False


In [18]:
# Counting and printing the number of users with too short passwords
# ... YOUR CODE FOR TASK 2 ...

print(users['too_short'].sum())

# Taking a look at the 12 first rows
# ... YOUR CODE FOR TASK 2 ...

print(users.head(n=12))

376
    id         user_name          password  password_length  too_short
0    1    vance.jennings          joobheco                8      False
1    2    consuelo.eaton        0869347314               10      False
2    3   mitchel.perkins        fabypotter               10      False
3    4    odessa.vaughan         aharney88                9      False
4    5    araceli.wilder        acecdn3000               10      False
5    6  shawn.harrington           5278049                7       True
6    7        evelyn.gay            master                6       True
7    8       noreen.hale            murphy                6       True
8    9       gladys.ward           lwsves2                7       True
9   10   brant.zimmerman  1190KAREN5572497               16      False
10  11     leanna.abbott          aivlys24                8      False
11  12   milford.hubbard           hubbard                7       True


## 3.  Common passwords people use
<p>Already this simple rule flagged a couple of offenders among the first 12 users. Next up in Special Publication 800-63B is the rule that</p>
<blockquote>
  <p>verifiers SHALL compare the prospective secrets against a list that contains values known to be commonly-used, expected, or compromised.</p>
  <ul>
  <li>Passwords obtained from previous breach corpuses.</li>
  <li>Dictionary words.</li>
  <li>Repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).</li>
  <li>Context-specific words, such as the name of the service, the username, and derivatives thereof.</li>
  </ul>
</blockquote>
<p>We're going to check these in order and start with <em>Passwords obtained from previous breach corpuses</em>, that is, websites where hackers have leaked all the users' passwords. As many websites don't follow the NIST guidelines and encrypt passwords there now exist large lists of the most popular passwords. Let's start by loading in the 10,000 most common passwords which I've taken from <a href="https://github.com/danielmiessler/SecLists/tree/master/Passwords">here</a>.</p>

In [38]:
# Reading in the top 10000 passwords
common_passwords = pd.read_csv(
    'https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/CommonCreds/10-million-password-list-top-10000.txt',
    sep=" ",
    header=None
)

common_passwords.columns = ['password']

# Taking a look at the top 20
#  YOUR CODE FOR TASK 3 
common_passwords.head(n=20)

,password
0,123456
1,password
2,12345678
3,qwerty
4,123456789
5,12345
6,1234
7,111111
8,1234567
9,dragon


In [44]:
commoners = common_passwords['password'].values

## 4.  Passwords should not be common passwords
<p>The list of passwords was ordered, with the most common passwords first, and so we shouldn't be surprised to see passwords like <code>123456</code> and <code>qwerty</code> above. As hackers also have access to this list of common passwords, it's important that none of our users use these passwords!</p>
<p>Let's flag all the passwords in our user database that are among the top 10,000 used passwords.</p>

In [45]:
# Flagging the users with passwords that are common passwords
users['common_password'] = users['password'].apply(lambda x: x in commoners)

# Counting and printing the number of users using common passwords
# ... YOUR CODE FOR TASK 4 ...

users['common_password'].sum()

129

In [46]:
# Taking a look at the 12 first rows
# ... YOUR CODE FOR TASK 4 ...

users.head()

,id,user_name,password,password_length,too_short,common_password
0,1,vance.jennings,joobheco,8,False,False
1,2,consuelo.eaton,0869347314,10,False,False
2,3,mitchel.perkins,fabypotter,10,False,False
3,4,odessa.vaughan,aharney88,9,False,False
4,5,araceli.wilder,acecdn3000,10,False,False


## 5. Passwords should not be common words
<p>Ay ay ay! It turns out many of our users use common passwords, and of the first 12 users there are already two. However, as most common passwords also tend to be short, they were already flagged as being too short. What is the next thing we should check?</p>
<blockquote>
  <p>Verifiers SHALL compare the prospective secrets against a list that contains [...] dictionary words.</p>
</blockquote>
<p>This follows the same logic as before: It is easy for hackers to check users' passwords against common English words and therefore common English words make bad passwords. Let's check our users' passwords against the top 10,000 English words from <a href="https://github.com/first20hours/google-10000-english">Google's Trillion Word Corpus</a>.</p>

In [50]:
# Reading in a list of the 10000 most common words
words = pd.read_csv(
    'https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english-no-swears.txt',
    sep=" ",
    header=None
)

words.columns = ['words']

# Taking a look at the top 20
#  YOUR CODE FOR TASK 3 
words.head(n=20)

,words
0,the
1,of
2,and
3,to
4,a
5,in
6,for
7,is
8,on
9,that


In [52]:
commoners = words['words'].values

In [53]:
# Flagging the users with passwords that are common words
users['common_word'] = users['password'].apply(lambda x: x in commoners)

# Counting and printing the number of users using common words as passwords
# ... YOUR CODE FOR TASK 5 ...
users['common_word'].sum()

134

In [54]:
# Taking a look at the 12 first rows
# ... YOUR CODE FOR TASK 5 ...

users.head(n=12)

,id,user_name,password,password_length,too_short,common_password,common_word
0,1,vance.jennings,joobheco,8,False,False,False
1,2,consuelo.eaton,0869347314,10,False,False,False
2,3,mitchel.perkins,fabypotter,10,False,False,False
3,4,odessa.vaughan,aharney88,9,False,False,False
4,5,araceli.wilder,acecdn3000,10,False,False,False
5,6,shawn.harrington,5278049,7,True,False,False
6,7,evelyn.gay,master,6,True,True,True
7,8,noreen.hale,murphy,6,True,True,True
8,9,gladys.ward,lwsves2,7,True,False,False
9,10,brant.zimmerman,1190KAREN5572497,16,False,False,False


In [55]:
'123.1231'.split('.')

['123', '1231']

## 6. Passwords should not be your name
<p>It turns out many of our passwords were common English words too! Next up on the NIST list:</p>
<blockquote>
  <p>Verifiers SHALL compare the prospective secrets against a list that contains [...] context-specific words, such as the name of the service, the username, and derivatives thereof.</p>
</blockquote>
<p>Ok, so there are many things we could check here. One thing to notice is that our users' usernames consist of their first names and last names separated by a dot. For now, let's just flag passwords that are the same as either a user's first or last name.</p>

In [71]:
# Extracting first and last names into their own columns
users['first_name'] = users['user_name'].apply(lambda x: x.split('.')[0])
users['last_name'] = users['user_name'].apply(lambda x: x.split('.')[1])

In [75]:
# Flagging the users with passwords that matches their names
users['uses_name'] = (users['password'] == users['first_name']) | (users['password'] == users['last_name'])

In [64]:
# Counting and printing the number of users using names as passwords
# ... YOUR CODE FOR TASK 6 ...

print(users['uses_name'].sum())

# Taking a look at the 12 first rows
# ... YOUR CODE FOR TASK 6 ...

users['uses_name'].head(n=12)

50


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11     True
Name: uses_name, dtype: bool

## 7. Passwords should not be repetitive
<p>Milford Hubbard (user number 12 above), what where you thinking!? Ok, so the last thing we are going to check is a bit tricky:</p>
<blockquote>
  <p>verifiers SHALL compare the prospective secrets [so that they don't contain] repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).</p>
</blockquote>
<p>This is tricky to check because what is <em>repetitive</em> is hard to define. Is <code>11111</code> repetitive? Yes! Is <code>12345</code> repetitive? Well, kind of. Is <code>13579</code> repetitive? Maybe not..? To check for <em>repetitiveness</em> can be arbitrarily complex, but here we're only going to do something simple. We're going to flag all passwords that contain 4 or more repeated characters.</p>

In [68]:
### Flagging the users with passwords with >= 4 repeats
users['too_many_repeats'] = users['password'].apply(lambda password: any(password.count(x) >= 4 for x in password))

# Taking a look at the users with too many repeats
# ... YOUR CODE FOR TASK 7 ...
users[users['too_many_repeats']].head()

,id,user_name,password,password_length,too_short,common_password,common_word,first_name,last_name,uses_name,too_many_repeats
38,39,gus.padilla,wwewwf1,7,True,False,False,gus,padilla,False,True
88,89,ira.cortez,mmommy22,8,False,False,False,ira,cortez,False,True
146,147,patti.dixon,555555,6,True,True,False,patti,dixon,False,True
180,181,pierre.hooper,10101971,8,False,False,False,pierre,hooper,False,True
192,193,trinidad.austin,012227257,9,False,False,False,trinidad,austin,False,True


## 8. All together now!
<p>Now we have implemented all the basic tests for bad passwords suggested by NIST Special Publication 800-63B! What's left is just to flag all bad passwords and maybe to send these users an e-mail that strongly suggests they change their password.</p>

In [78]:
# Flagging all passwords that are bad
users['bad_password'] = (users['too_many_repeats']) | (users['uses_name']) | (users['common_word']) | (users['common_password'])

# Counting and printing the number of bad passwords
# ... YOUR CODE FOR TASK 8 ...

users['bad_password'].sum()

# Looking at the first 25 bad passwords
# ... YOUR CODE FOR TASK 8 ...

users[users['bad_password']].head(25)

,id,user_name,password,password_length,too_short,common_password,common_word,first_name,last_name,uses_name,too_many_repeats,bad_password
6,7,evelyn.gay,master,6,True,True,True,evelyn,gay,False,False,True
7,8,noreen.hale,murphy,6,True,True,True,noreen,hale,False,False,True
11,12,milford.hubbard,hubbard,7,True,False,False,milford,hubbard,True,False,True
16,17,domingo.dyer,chelsea,7,True,True,True,domingo,dyer,False,False,True
21,22,leticia.sanford,cocacola,8,False,True,False,leticia,sanford,False,False,True
22,23,jenny.woodard,woodard,7,True,False,False,jenny,woodard,True,False,True
30,31,rosanna.reid,reid,4,True,False,True,rosanna,reid,True,False,True
38,39,gus.padilla,wwewwf1,7,True,False,False,gus,padilla,False,True,True
51,52,ahmad.hopper,123456789,9,False,True,False,ahmad,hopper,False,False,True
58,59,rosalinda.rodriquez,golden,6,True,True,True,rosalinda,rodriquez,False,False,True


## 9. Otherwise, the password should be up to the user
<p>In this notebook, we've implemented the password checks recommended by the NIST Special Publication 800-63B. It's certainly possible to better implement these checks, for example, by using a longer list of common passwords. Also note that the NIST checks in no way guarantee that a chosen password is good, just that it's not obviously bad.</p>
<p>Apart from the checks we've implemented above the NIST is also clear with what password rules should <em>not</em> be imposed:</p>
<blockquote>
  <p>Verifiers SHOULD NOT impose other composition rules (e.g., requiring mixtures of different character types or prohibiting consecutively repeated characters) for memorized secrets. Verifiers SHOULD NOT require memorized secrets to be changed arbitrarily (e.g., periodically).</p>
</blockquote>
<p>So the next time a website or app tells you to "include both a number, symbol and an upper and lower case character in your password" you should send them a copy of <a href="https://pages.nist.gov/800-63-3/sp800-63b.html">NIST Special Publication 800-63B</a>.</p>

In [0]:
# Enter a password that passes the NIST requirements
# PLEASE DO NOT USE AN EXISTING PASSWORD HERE
new_password = "qwerty123"